## UM-Bridge Integration for Predator-Prey Model

### First Client

Now the server is running (see `predprey.model.ipynb`) and waits for requests by a client. Before you can make a model evaluation you have to connect to the model.

In [12]:
import umbridge
model = umbridge.HTTPModel("http://0.0.0.0:4242", "forward")

Then you have to fix the model parameters $\theta_1, \theta_2, \theta_{12}, \theta_{21}$, choose a time point $t$ and set the inital values for prey $\textbf{u}_0[0]$ and predator $\textbf{u}_0[1]$. You can try different points in time, initial conditions or parameters and see how the system changes.

In [13]:
# parameters = [t, u0[0], u0[1], theta1, theta2, theta12, theta21]
parameters = [6, 0.5, 2, 1, 1, 1, 1]
print(model([parameters]))

[[1.4098300518932472, 2.2523325849320948]]


### Client for MC simulation
In the following you can see the client code corresponding to the MC simulation in `predprey_UQ.ipynb`.

In [15]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import umbridge

# Connect to the umbridge model
model = umbridge.HTTPModel("http://0.0.0.0:4242", "forward")

# Define parameters
theta1, theta2, theta12, theta21 = 1.0, 1.0, 1.0, 1.0
T = 6
u0_bar = np.array([0.5, 2])  # Example unperturbed initial condition

# Generate perturbed initial conditions
delta = 0.2
N = 1000  # Number of MC samples

np.random.seed(42)

initial_conditions = u0_bar + delta * (2 * np.random.rand(N, 2) - 1)

# Solve ODE for each initial condition using the umbridge model
u1_T_values = []  # List for storing values of u1(T), i.e., prey, at T=6
for u0 in initial_conditions:
    # Create the input for the model
    parameters = [T, u0[0], u0[1], theta1, theta2, theta12, theta21]
    result = model([parameters])
    u1_T_values.append(result[0][0])  # Extract u1(T) from the result

# Calculate MC estimator
u1_T_mean = np.mean(u1_T_values)
print(f"MC estimator for expected value of u1(T) at T={T}: {u1_T_mean}")

# Solve ODE for the unperturbed initial condition using the umbridge model
parameters_unperturbed = [T, u0_bar[0], u0_bar[1], theta1, theta2, theta12, theta21]
result_unperturbed = model([parameters_unperturbed])
u1_T_unperturbed = result_unperturbed[0][0]
print(f"Value of u1(T) for the unperturbed trajectory for T=6: {u1_T_unperturbed}")

MC estimator for expected value of u1(T) at T=6: 1.5181587659330416
Value of u1(T) for the unperturbed trajectory for T=6: 1.4098300518932472
